In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
from zero_helpers.imports import * 
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

# Run Simulation

In [ ]:
cfg = {
    "git_commit": "c440da9b1f1dee0556f93d8843a873e5dabc0002",
    "SimulationManager": {
        "__init__": {
            "n_agents": 30,
            "lift_capacity": 2,
            "start_datetime": None,
            "logger": None
        },
        "run": {
            "end_datetime": None,
            "runtime_minutes": None
        }
    },
    "Simulation": {
        "__init__": {
            "simulation_uuid": None
        },
        "run": {
            "stop_time": None,
            "logger": None,
            "full_agent_logging": False,
            "start_datetime": "2025-03-12T09:00:00",
            "random_seed": None
        }
    },
    "Lift": {
        "num_chairs": 15,
        "ride_mean": 7,
        "ride_sd": 1,
        "traverse_mean": 5,
        "traverse_sd": 1.5
    },
    "Agent": {
        "__init__": {
            "logger": None,
            "self_logging": True
        }
    }
}
cfg['Simulation']['run']['random_seed'] = '46344237-3141-4d05-9b8f-4854a7b47c2f'

manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60*5)

## Retrieve Data from Simulation

In [57]:
# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

In [59]:
exp.columns

Index(['time', 'exp_id', 'return_event_uuid', 'agent_uuid',
       'agent_uuid_codename', 'time_spent_ride_lift',
       'time_spent_traverse_down_mountain', 'time_spent_in_queue',
       'time_spent_total'],
      dtype='object')

In [63]:
exp.iloc[227]['time_spent_in_queue']

2

In [65]:
exp.dtypes

time                                 datetime64[ns]
exp_id                                       object
return_event_uuid                            object
agent_uuid                                   object
agent_uuid_codename                          object
time_spent_ride_lift                         object
time_spent_traverse_down_mountain            object
time_spent_in_queue                          object
time_spent_total                             object
dtype: object

In [49]:
exp['time_spent_in_queue'].astype(float).describe()

count    748.000000
mean       0.524064
std        0.715638
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        5.000000
Name: time_spent_in_queue, dtype: float64

In [51]:
print(exp.columns)

Index(['time', 'exp_id', 'return_event_uuid', 'agent_uuid',
       'agent_uuid_codename', 'time_spent_ride_lift',
       'time_spent_traverse_down_mountain', 'time_spent_in_queue',
       'time_spent_total'],
      dtype='object')


In [52]:
print(log.columns)

Index(['return_event_uuid', 'time', 'time_offset', 'event', 'agent_id',
       'agent_uuid', 'agent_uuid_codename', 'queue_length', 'lift_state',
       'wait_time', 'wait_time_readable'],
      dtype='object')


# Scratchpad

In [61]:
raise Exception()

Exception: 

## Sample Agent and Get Exp Traceback

In [13]:
# sample agent and subset exp/log to e/l for agent subset of those
m = log['event'] == 'ride_complete'
event_ride_comp = log[m].sample().iloc[0].to_dict()
agent = manager.lookup_agent(event_ride_comp['agent_uuid'])
l = log[log['agent_uuid'] == agent.agent_uuid]
idx = (exp['time'] - event_ride_comp['time']).dt.total_seconds().abs().idxmin()
e = exp.loc[idx].to_dict()

In [14]:
x = agent.traceback_experience(e['exp_id'])
print(x)

# Experience Log 

exp_id: 4eab1919-d47e-4c6a-9adc-43f9eee18e5c
agent_uuid: 27596a10-a9ac-48ee-aa91-c96421db4589
return_event_uuid: 961742d5-cd10-48cd-b712-5bf11cd2524e

*** 


time_spent_in_queue: 3
time_spent_ride_lift: 5.261610974166602
time_spent_traverse_down_mountain: 5.481852237108882

***

# Full Dict

{
    "exp_id": "4eab1919-d47e-4c6a-9adc-43f9eee18e5c",
    "return_event_uuid": "961742d5-cd10-48cd-b712-5bf11cd2524e",
    "agent_uuid": "27596a10-a9ac-48ee-aa91-c96421db4589",
    "agent_uuid_codename": "abounding-basket",
    "time_spent_ride_lift": 5.261610974166602,
    "time_spent_traverse_down_mountain": 5.481852237108882,
    "time_spent_in_queue": 3,
    "time_spent_total": 13.743463211275483
}

# Agent Log

{
    "787aebe4-4960-4d9f-8c3a-54ff0f20fcef": {
        "return_event_uuid": "cf7e394b-044c-4276-9616-42961b14cbd9",
        "time": "2025-03-12T09:24:00",
        "time_offset": 24,
        "event": "ride_complete",
        "agent_id": 3,
        "agent_uuid": "275